# Demo: encoding apollo scape

## Step 1: Loading all meshes from Apollo Scape

In [ ]:
import glob
import torch
from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.vis.plotly_vis import plot_scene
from pprint import pprint

TSDF_UNIT = 0.1
APOLLO_SCAPE_PATH = "../assets/apollo_scape/*.obj"

obj_glob = glob.glob(APOLLO_SCAPE_PATH)
pprint(obj_glob)

apollo_scape_meshes = None

verts_list, faces_list = list(), list()
for obj_path in obj_glob:
    verts, faces, _ = load_obj(obj_path, load_textures=False)
    verts_list.append(verts)
    faces_list.append(faces.verts_idx)

apollo_scape_meshes = Meshes(verts_list, faces_list)
print(f"Meshes size: {apollo_scape_meshes.__len__()}")

## Step 2: Preparing all the meshes

In [ ]:
bbox = apollo_scape_meshes.verts_packed().amax(0) - apollo_scape_meshes.verts_packed().amin(0)
antisotropic_res = (bbox / TSDF_UNIT).ceil().int()
print(f"bbox: {bbox}")
print(f"resolution: {antisotropic_res}")

# Move all the vehicle to the center
apollo_scape_meshes.offset_verts_(torch.Tensor([0.0, -bbox[1].item()/2, 0.0]))

## Step 3: Instantiating TSDF object & run

In [ ]:
import sys
import os

sys.path.append("..")

from voxeltorch import TSDF, tsdf2meshes

tsdf = TSDF(resolution=antisotropic_res, sampling_count=4096, downsampling_count=2048, bbox=bbox, isotropic=True)

In [ ]:
tsdf_grid = tsdf.tsdf(apollo_scape_meshes)
print(tsdf_grid.size())

## Step 4: Visualizing TSDF Meshes

In [ ]:
def visualize_meshes(meshes:Meshes, title:str="Mesh"):
    mesh_dict = { }
    for idx in range(meshes.__len__()):
        mesh_dict[title + f" {idx}"] = {
            "mesh": meshes[idx]
        }
    fig = plot_scene(mesh_dict, ncols=2)
    fig.update_layout(height=400, width=400 * meshes.__len__())
    return fig.show()

In [ ]:
selected_idx = torch.randint(0, apollo_scape_meshes.__len__() - 1, (2, ))
print("Visualizing original meshes...")
visualize_meshes(apollo_scape_meshes[selected_idx], "Original Meshes")
print("Visualizing voxel meshes from TSDF...")
tsdf_meshes = tsdf2meshes(tsdf_grid[selected_idx], antisotropic_res / bbox)
visualize_meshes(tsdf_meshes, "TSDF Meshes")